In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [3]:
#고급 차량 관련 문서로 테스트 진행
doc = df_add[1]


In [4]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

#구두점 제거
doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()
print(len(doc1))

#숫자 제거
doc2 = "".join([i for i in doc1 if not i.isdigit()])
print(len(doc2))

#월 제거
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
         'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   
    
doc3 = " ".join([i for i in doc2.split() if i not in month])
print(len(doc3))

# word_tokens = word_tokenize(doc2)

#동사 명사
# word_tokens = pos_tag(word_tokenize(doc))
# word_tokens = [t[0] for t in word_tokens if t[1] == "NN"]

1815
1710
1653


In [5]:
n_gram_range = (2,2)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc3])
candidates = count.get_feature_names_out()

print('bigram 개수 :',len(candidates))
print('bigram 출력 :',candidates)

bigram 개수 : 125
bigram 출력 : ['abfgovau amendments' 'additional duty' 'additional note'
 'alterations notice' 'amendments online' 'applied implementation'
 'applies goods' 'apply refund' 'attachment notice' 'attachment online'
 'australian notice' 'available attachment' 'available current'
 'belarus currently' 'car tax' 'cargo ics' 'cargo new' 'cent applied'
 'certain electric' 'chapter new' 'circumstance eb'
 'classifications available' 'classifications commence'
 'classifications entered' 'commence retrospectively'
 'consistent australian' 'consumption lct' 'consumption time'
 'current working' 'currently additional' 'customs duty'
 'directed tradepolicyabfgovau' 'duty applies' 'duty cent' 'duty certain'
 'duty electric' 'duty free' 'eb quoted' 'efficient luxury'
 'efficient vehicles' 'electric vehicles' 'entered home' 'entered ics'
 'entered integrated' 'financial year' 'following publication' 'free rate'
 'fuel efficient' 'fuelcell vehicles' 'goods entered' 'goods meet'
 'home consu

In [7]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc]) #전체 문서
candidate_embeddings = model.encode(candidates)#추출한 bigram

Max Sum Similarity

In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [9]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['hydrogen fuelcell',
 'hybrid vehicles',
 'tariff alterations',
 'new tariff',
 'car tax']

Maximal Marginal Relevance

In [10]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [11]:
#diversity 낮게 설정하면, 기존의 코사인 유사도만 사용한 것과 매우 유사함
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['tax threshold',
 'hybrid vehicles',
 'online tariff',
 'new tariff',
 'russia belarus']

In [12]:
#but 높은 diversity는 다양한 키워드 추출
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['tax threshold',
 'russia belarus',
 'fuel efficient',
 'hydrogen fuelcell',
 'following publication']